# CUDA + Python

## Using Numba for Pythonic GPU-accelerated Code

"**Numba** is a just-in-time compiler for Python functions."

In addition to its primary function, Numba supports CUDA GPU programming by directly compiling a **restricted subset of Python code** into CUDA kernels and device functions, following the CUDA execution model.

As discussed in class, there are two important points to keep in mind when working with code that runs on GPUs:
1. Data required for kernel execution must be transferred between the host and the device
2. GPU kernels will not provide performance benefits over CPU code unless the full bandwidth of the GPU is utilized effectively

In this notebook, we will explore how to program a GPU using Python, write CUDA kernels, and manage data exchange between the host and device.

We will start with what we covered last on Numba: the `@vectorize` and `@guvectorize` decorators. These decorators allow us to create compiled functions that can target multiple execution environments, including `CPU`, multithreaded `parallel` execution, and `CUDA` for GPU acceleration.

But first, let's use the `numba.cuda` API to detect and list the available CUDA devices. This will help us ensure that we have the necessary hardware for GPU programming and allow us to verify the device's capabilities before we start writing CUDA kernels.

In [1]:
from numba import cuda, vectorize, guvectorize

We can check if any CUDA-capable GPU is available by using the `cuda.is_available()` function.

In [ ]:
# Verify availability of CUDA
cuda_available = cuda.is_available()
cuda_available

Next, we can detect the supported CUDA hardware by using the `cuda.detect()` function. This function will provide a summary of the available CUDA devices and their capabilities, helping us understand the specifications of the hardware we will be using for GPU programming.

In [ ]:
# Display the summary information of the available CUDA devices
cuda.detect()

## Writing ufuncs for the GPU with `@vectorize`

Let's redefine a simple `vectorize` universal function, explicitly stating that it will be executed as a kernel on the GPU.

It's important to note that, in order to run on a GPU, the CUDA `@vectorize` and `@guvectorize` decorators will not behave identically to the NumPy `ufunc` produced by standard Numba. We need to be aware of these differences as we write GPU-accelerated functions.

In [4]:
# Define a vectorized universal function for execution on --CPU--
@vectorize(['int64(int64, int64)',
            'int32(int32, int32)',
            'float64(float64, float64)',
            'float32(float32, float32)'], target='cpu') 
def residual(x, y):
    return (y - x) / x

In [5]:
# Define a vectorized universal function for execution on --GPU--
# In this case, the signature and target types are explicitely required
@vectorize(['int64(int64, int64)',
            'int32(int32, int32)',
            'float64(float64, float64)',
            'float32(float32, float32)'], target='cuda') 
def residual_cu(x, y):
    return (y - x) / x

The inputs and outputs for the `@vectorize` decorator are specified as a list of signatures, allowing us to compile the same universal function (ufunc) for multiple data types on the GPU.

The CUDA `ufunc` also supports passing arrays that are already on the GPU device (we will explore this later). However, it can also accept arrays that reside on the host. 

Numba automatically manages the transfer of data between the host and the device during the function call, simplifying the programming process for GPU acceleration.

In [ ]:
# Import NumPy for array creation
import numpy as np

# Create two random arrays 
a = np.random.random(100_000)
b = np.random.random(100_000)

# Check the data type of the array 'a'
a.dtype

In [ ]:
%%timeit -n 5 -r 5

# Calculate the residuals using NumPy operations
c = (b - a) / a

In [ ]:
%%timeit -n 5 -r 5

# Calculate the residuals using the CPU implementation
c = residual(a,b)

In [ ]:
%%timeit -n 5 -r 5

# Calculate the residuals using the GPU implementation
c = residual_cu(a,b)

In this simple call to the GPU `@vectorize` function, Numba automatically handles several important tasks:

- Compiled the CUDA kernel.
- (!) Allocated GPU memory for the inputs and outputs.
- (!) Copied the input data to the GPU.
- (!) Executed the CUDA kernel (GPU function) with the appropriate kernel dimensions based on the input sizes.
- (!) Copied the result back from the GPU to the CPU.
- Returned the result as a NumPy array on the host.

It's a beautifully simple and compact piece of code that performs all the actions we used to execute explicitly in CUDA-C with almost zero effort.

***BUT...*** the performance may not meet expectations (i.e. may be crappy).

Based on our experience with CUDA-C, there are several potential root causes for subpar performance:

- **Input Size**: The inputs may be too small for the GPU, which is designed for throughput rather than low-latency operations. This can lead to insufficient performance given the hardware differences between CPU cores and GPU SPs.
  
- **Simple Calculations**: The calculation being performed may be too simple. Sending a task to the GPU incurs significant overhead compared to executing a function on the CPU. If our Compute operations to Global Memory Accesses (CGMA) ratio is too low, the latency associated with global memory access can dominate the computation time.

- **Data Transfer Overhead**: The `timeit` measurement also includes the time taken to copy data to and from the GPU. While Numba simplifies data management for a single function call, it is often more efficient to run multiple GPU operations sequentially, transferring data to the GPU once and keeping it there until all processing is complete.

- **Data Types**: Our data types might not be optimal for the task at hand, potentially being larger than necessary. For example, using `float64` instead of `float32` when precision requirements allow for a smaller data type can lead to inefficiencies (double the size of data moved and accessed).

We have extensively discussed that maximizing the performance of our GPU computations requires more than just dispatching the same code to the GPU. It involves rethinking algorithms, estimating the relationship between data size and computational complexity, and implementing careful memory management.

In [9]:
import math

# Precompute this constant as a float32
SQRT_2PI = np.float32((2 * math.pi) ** 0.5)

# Define a vectorized function to compute the values of a Gaussian on the GPU
@vectorize(['float32(float32, float32, float32)'], 
           target='cuda')
def gaussian_cu(x, m, s):
    return math.exp(-0.5 * ((x - m) / s) ** 2) / (s * SQRT_2PI)

# Define a function to compute the values of a Gaussian with NumPy
def gaussian_np(x, m, s):
    return np.exp(-0.5 * ((x - m) / s) ** 2) / (s * SQRT_2PI)

In [10]:
# Prepare to evaluate the Gaussian function a few million times with the same mean and standard deviation
x = np.random.uniform(-5, 5, size=10_000_000).astype('float32')
mean = np.float32(0.0)
sigma = np.float32(1.0)

In [ ]:
%%timeit -n 5 -r 5

# Compute the Gaussian values with NumPy
gaussian_np(x, mean, sigma)

In [ ]:
%%timeit -n 5 -r 5

# Compute the Gaussian values with Numba+CUDA
gaussian_cu(x,mean,sigma)

As expected, we are observing performance improvements with the following optimizations:

- **Using `float32` Type**: Leveraging `float32` instead of `float64` reduces memory usage and increases computation speed.
  
- **More Computation per Memory Access**: The calculations performed within the GPU function provide a higher computation-to-memory access ratio, which is essential for maximizing performance.

- **High Concurrency**: A large number of concurrent GPU threads are active during execution, further enhancing throughput.

It's important to note that the timing for the GPU function still includes the overhead associated with copying all the data to and from the GPU. This overhead can impact overall performance, particularly for smaller workloads.

> Why are we using `math.exp` instead of `np.exp`?
> 
> We use `math.exp` instead of `np.exp` due to the limitations of Numba's CUDA support for NumPy functions. 
> 
> Depending on the version of Numba you have installed, you may often need to use scalar Python functions instead of their NumPy equivalents when targeting CUDA. While `np.exp` (and other NumPy functions) may work in more recent or future versions of Numba, current versions may require this specific approach.
> 
> Given our current hardware and software configuration, we are using a specific combination of packages:
> ```bash
> # Name                    Version                   Build  Channel
> python                    3.9.10       h1b383ca_2_cpython    conda-forge
> numpy                     1.23.5           py39hf5a3166_0    conda-forge
> numba                     0.56.4           py39h6619693_1    conda-forge
> ```
> 
> It's worth noting that recent versions of Numba have included compatibility with many more NumPy functions.
> 
> As always, checking the documentation for the specific versions we use it's paramount... [https://numba.readthedocs.io/en/stable/cuda/cudapysupported.html](https://numba.readthedocs.io/en/stable/cuda/cudapysupported.html)

## Writing ufuncs for the GPU with `@guvectorize`

The same considerations apply to functions that operate on entire ndarrays using `@guvectorize`.

For example, let's rewrite the matrix element-wise addition code using Numba and CUDA with the `@guvectorize` decorator:

In [14]:
# Define the signature for the guvectorize function
# and the layouts of the input and output
# 2 matrices (A, B) as input and 1 matrix (C) as output
# All elements are of type float32
@guvectorize('(float32[:,:], float32[:,:], float32[:,:])',
             '(x,y),(x,y)->(x,y)', 
             target='cuda')
def matrix_addition(A, B, C):
    # Loop over each element of the 2D arrays
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            # Perform element-wise addition
            C[i, j] = A[i, j] + B[i, j]

In [15]:
# Define the size of the matrices
rows, cols = 1024, 1024

# Create two random matrices A and B with float32 elements
A = np.random.rand(rows, cols).astype(np.float32)
B = np.random.rand(rows, cols).astype(np.float32)

# Create an empty matrix for the result C
C = np.zeros((rows, cols), dtype=np.float32)

In [ ]:
%%timeit -n 1 -r 1

# Perform the matrix addition using the guvectorized function
matrix_addition(A, B, C)

In [ ]:
# Print the resulting matrix
C

> Remember that `@guvectorize` does not allow returning the result directly. Instead, the result object must be passed as an input parameter. This design choice emphasizes the need for explicit memory management in GPU programming, ensuring that the output is written to a predefined location in memory.

## Memory Management

So far, we have operated directly with NumPy arrays on the host. 

During the kernel call, Numba automatically handles the transfer of these arrays to the device and back to the host after execution. While this convenience is useful, it is not particularly efficient.

In most cases, you will want to keep data on the GPU and launch multiple kernels without being interrupted by transfers between the device and the host.

We can utilize the CUDA APIs for manual data management, allowing for more efficient handling of memory on the GPU:

- `cuda.device_array`: Allocates memory on the device.
- `cuda.to_device`: Allocates memory on the device and, by default (`copy=True`), copies data from an existing host array.
- `cuda.copy_to_host`: Transfers data from the device memory back to the host.

Now, let's return to the previous example and see how we can apply these concepts:

In [18]:
# Define the size of the matrices
rows, cols = 1024, 1024

# Create host arrays A and B with random float32 values
A = np.random.rand(rows, cols).astype(np.float32)
B = np.random.rand(rows, cols).astype(np.float32)

In [ ]:
# Check if A is a CUDA array
# (an array managed by the CUDA backend)
cuda.is_cuda_array(A)

In [20]:
# Create device arrays and perform a host-to-device copy
# Both operations are done in a single call with `to_device()`
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)

# Allocate a device array with the same shape and type as d_A
d_C = cuda.device_array_like(d_A)

In [ ]:
# Check if d_A is a CUDA array
cuda.is_cuda_array(d_A)

In [ ]:
# Print the shape and memory allocation details for the device array d_A
print(f'Shape of array d_A in device memory: {d_A.shape}')
print(f'Bytes allocated for d_A: {d_A.nbytes / 1e6:.1f} MB')

In [ ]:
%%timeit -n 3 -r 3

# Perform the matrix addition using the previous guvectorized funtion but with device memory
matrix_addition(d_A, d_B, d_C)

Note that this time measurement is now completely bogus!

The launch of a kernel is an asynchronous operation that does not block the CPU. As a result, the time we measured does not accurately reflect the execution time of the task on the GPU.

To obtain a reliable measurement, we should revert to using CUDA Events for timing. CUDA Events allow us to record timestamps before and after the kernel execution, enabling us to measure the actual GPU execution time more accurately.

In [ ]:
# Create the CUDA events for timing
start = cuda.event()
stop = cuda.event()

# Start the timer
start.record()

# Perform the matrix addition using the device memory
matrix_addition(d_A, d_B, d_C)

# Stop the timer and synchronize it
stop.record()
stop.synchronize()

# Get the elapsed time between start and stop (in milliseconds)
print(f"Elapsed time: {cuda.event_elapsed_time(start, stop):.1f} ms")

In [ ]:
# Copy the result from the device back to the host
C = d_C.copy_to_host()

# Display the result
C

We discussed that it is very important to free up memory on the device by deallocating the memory used in previous kernel executions.

However, it's essential to note that Python is not `C` or `C++`. An analogous call to `cudaFree` will not have an immediate effect when issued.

Python is a garbage-collected language, meaning that memory deallocation will be handled by the Python interpreter at some point in the future... eventually...

While it is always good practice to deallocate memory, the underlying logic in Python is less straightforward compared to memory-managed languages like `C` or `C++`. Therefore, we must be mindful of how and when we manage device memory to avoid potential memory leaks or unnecessary memory consumption.

In [27]:
# Delete (free?) the device-allocated arrays
del d_A, d_B, d_C

What Python did with this command is remove the references to the `d_A`, `d_B`, and `d_C` objects, but it does not immediately free up the memory they occupy.

Numba will take care of releasing the memory when the garbage collector runs, which may not happen right away.

We don't need to know the intricate details of how this works, but it's important to understand that this is one of the reasons why GPU code is typically written in memory-managed languages like `C` or `C++`. In those languages, developers have direct control over memory allocation and deallocation, allowing for more predictable memory management.

## Expressing CUDA Kernels with `@jit`

In addition to creating kernels as `@vectorize` NumPy-like functions, we can use `@jit` or `@njit` to express entire functions targeted at the GPU instead of the CPU.

Let’s revisit the creation of the Julia fractal, this time rewriting the Numba code to execute as a CUDA kernel.

To do this, we need to access the "location" of the kernel during execution on the grid. The `numba.cuda` module provides access to these registers, similar to what we see in CUDA-C.

For a 1D grid of blocks and threads, you can determine the unique index for each thread using the following code:

```python
tx = cuda.threadIdx.x
bx = cuda.blockIdx.x
bw = cuda.blockDim.x
idx = tx + bx * bw
```

This allows you to access each element of your data using:

```python
array[idx] = ...
```

Similarly, for a 2D grid of blocks and threads, one can use: 
```python
tx = cuda.threadIdx.x
ty = cuda.threadIdx.y

bx = cuda.blockIdx.x
by = cuda.blockIdx.y

bw = cuda.blockDim.x
bh = cuda.blockDim.y

idx_x = tx + bx * bw
idx_y = ty + by * bh

[...]

array[idx_x, idx_y] = ...

```


Additionally, `numba.cuda` offers a simple API `cuda.grid(ndim)`  access the unique identifier of the thread in a 1D or 2D grid by calling:

```python
idx = cuda.grid(1)
```

or

```python
idx_x, idx_y = cuda.grid(2)
```

These functions return the absolute position of the current thread within the entire grid of blocks. The parameter `ndim` should match the number of dimensions specified when instantiating the kernel. If `ndim` is 1, a single integer is returned. If `ndim` is 2 or 3, a tuple containing the respective number of integers is returned.


Similarly, we can check the dimension of the grid by means of `numba.cuda.gridsize()`, which will return the absolute size (or shape) in threads of the entire grid of blocks. `ndim` has the same meaning as in `grid()` above.

### Re-computing the Julia Set with `cuda.jit`

Now, let's rewrite the equivalent plain-Numba code that was used to evaluate the Julia set, this time utilizing `cuda.jit` for CUDA execution in Numba.

In [28]:
@cuda.jit
def julia_fractal(z_re, z_im, j):
    # Get the position of the thread in the overall grid
    idx_x, idx_y = cuda.grid(2)

    """ This is equivalent to writing 
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    
    idx_x = tx + bx * bw

    ty = cuda.threadIdx.y
    by = cuda.blockIdx.y
    bh = cuda.blockDim.y

    idx_y = ty + by * bh
    """    

    # Ensure the thread is within the bounds of the data
    if idx_x < j.shape[0] and idx_y < j.shape[1]:
        # Initialize the complex number z
        z = z_re[idx_x] + 1j * z_im[idx_y]

        # Iterate to check the divergence
        for t in range(256):
            z = z ** 2 - 0.05 + 0.68j
            if (z.real * z.real + z.imag * z.imag) > 4.0:
                j[idx_x, idx_y] = t
                break

In [29]:
# Define grid dimensions; in this case, a square NxN
N = 1024
width, height = np.int32(N), np.int32(N)

# Create the arrays for the real and imaginary parts
z_real = np.linspace(-1.5, 1.5, width).astype('float32')
z_imag = np.linspace(-1.5, 1.5, height).astype('float32')

# Prepare the output array 
# Using uint8 since we only need to store values between 0 and 255
j = np.zeros((width, height), dtype=np.uint8)

In [30]:
# Create device arrays and perform a host-to-device copy (all at once) 
d_z_real = cuda.to_device(z_real)
d_z_imag = cuda.to_device(z_imag)
d_j      = cuda.to_device(j)

In [31]:
# Define the CUDA grid dimensions
threads_per_block = (8, 8)  # 8x8 threads per block
blocks_per_grid_x = math.ceil(j.shape[0] / threads_per_block[0])
blocks_per_grid_y = math.ceil(j.shape[1] / threads_per_block[1])

# Alternatively, you can use:
# blocks_per_grid_x = (j.shape[0] + threads_per_block[0] - 1) // threads_per_block[0]
# blocks_per_grid_y = (j.shape[1] + threads_per_block[1] - 1) // threads_per_block[1]

blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

In [ ]:
# Create the CUDA events
start = cuda.event()
stop = cuda.event()

# Start the timer
start.record()

# Launch the CUDA kernel
julia_fractal[blocks_per_grid, threads_per_block](d_z_real, d_z_imag, d_j)

# Stop the timer and synchronize it
stop.record()
stop.synchronize()

# Get the elapsed time between start and stop (in milliseconds)
print(f"Elapsed time: {cuda.event_elapsed_time(start, stop):.1f} ms")

In [ ]:
# Copy the result back to the host, storing it in the existing array j
d_j.copy_to_host(j)

In [ ]:
import matplotlib.pyplot as plt

# Display the Julia set 
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(j, cmap=plt.cm.RdBu_r, extent=[-1.5, 1.5, -1.5, 1.5])
ax.set_xlabel("$\\mathrm{Re}(z)$", fontsize=18)
ax.set_ylabel("$\\mathrm{Im}(z)$", fontsize=18)

plt.show()

In [35]:
# Attempt to free up the memory on the device
del d_z_real, d_z_imag, d_j

### Shared Memory and Thread Synchronization in Numba CUDA Kernels

As we discussed in CUDA-C, optimization of CUDA kernel execution can be achieved by reducing latency associated with global memory access and instead utilizing shared memory. Shared memory is located locally within the Streaming Multiprocessors (SMPs) and can be accessed by all threads within the same block.

Numba provides the capability to express the use of shared memory in CUDA by using:

```python
cuda.shared.array(shape, type)
```

This call should be placed inside the `cuda.jit` kernel, where memory coalescing is beneficial.

To synchronize the shared memory across all threads in a block, it is essential to include appropriate calls to `cuda.syncthreads()`.

Let's rewrite the _square-matrix multiplication with tiling_ CUDA-C example in Python with Numba+CUDA.

For the sake of comparing and porting CUDA-C code to Python, we'll work with the previous CUDA-C example which is limited to only allow square matrices with a width which is proportional to the tile.

```c
#define WIDTH 2048                      
#define TILE_WIDTH 32                   
#define THREADS_PER_BLOCK_X TILE_WIDTH  
#define THREADS_PER_BLOCK_Y TILE_WIDTH  

[...]


__global__ void matrixMultiplication(const float* M, const float* N, float* P, const int width) {
    __shared__ float M_tile[TILE_WIDTH][TILE_WIDTH];
    __shared__ float N_tile[TILE_WIDTH][TILE_WIDTH];
    
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = blockIdx.y * TILE_WIDTH + ty;
    int col = blockIdx.x * TILE_WIDTH + tx;

    float sum = 0.;

    // Fill the shared memory
    // Loop over the tiles of the input matrices
    for (int t = 0; t < width / TILE_WIDTH; ++t) {
        if ( (row < width) && (t * TILE_WIDTH + tx < width) )
            M_tile[ty][tx] = M[row * width + t * TILE_WIDTH + tx];
        else 
            M_tile[ty][tx] = 0.;

        if ( (t * TILE_WIDTH + ty < width) && (col < width) )
            N_tile[ty][tx] = N[(t * TILE_WIDTH + ty) * width + col];
        else 
            N_tile[ty][tx] = 0.;

        // Synchronize (ensure the tile is loaded in shared memory)
        __syncthreads();
    
        // Perform the multiplication for this tile
        for (int k = 0; k < TILE_WIDTH; ++k) {
            sum += M_tile[ty][k] * N_tile[k][tx];
        }

        // Ensure all threads are done computing before loading the next tile
        __syncthreads(); 
    }

    // Write the result back to the global memory
    if (row < width && col < width) {
        P[row * width + col] = sum;
    }
}
```

In [36]:
# Define constants for matrix dimensions and tile size
WIDTH = 2048 * 2                      
TILE_WIDTH = 32                   

# Set number of threads per block in both dimensions
THREADS_PER_BLOCK_X = TILE_WIDTH  
THREADS_PER_BLOCK_Y = TILE_WIDTH  

@cuda.jit
def matrix_multiplication(M, N, P):
    # Allocate shared memory for tiles
    M_tile = cuda.shared.array(shape=(TILE_WIDTH, TILE_WIDTH), dtype='float32')
    N_tile = cuda.shared.array(shape=(TILE_WIDTH, TILE_WIDTH), dtype='float32')

    # Thread index within the block
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    # Calculate the row and column indices for the global matrix
    row = cuda.blockIdx.y * TILE_WIDTH + ty
    col = cuda.blockIdx.x * TILE_WIDTH + tx

    # Initialize sum for the resulting matrix element
    sum = np.float32(0.)

    # Loop over the tiles of the input matrices
    for t in range(M.shape[0] // TILE_WIDTH):
        # Load tile from matrix M into shared memory
        if (row < WIDTH) and (t * TILE_WIDTH + tx < WIDTH):
            M_tile[ty][tx] = M[row][t * TILE_WIDTH + tx]
        else:
            M_tile[ty][tx] = 0.

        # Load tile from matrix N into shared memory
        if (t * TILE_WIDTH + ty < WIDTH) and (col < WIDTH):
            N_tile[ty][tx] = N[t * TILE_WIDTH + ty][col]
        else:
            N_tile[ty][tx] = 0.

        # Synchronize threads to ensure all data is loaded
        cuda.syncthreads()

        # Compute partial sum for the current tile
        for k in range(TILE_WIDTH):
            sum += M_tile[ty][k] * N_tile[k][tx]
        
        # Synchronize threads before loading the next tile
        cuda.syncthreads()
    
    # Write the result to the output matrix P
    if (row < WIDTH) and (col < WIDTH):
        P[row][col] = sum

In [37]:
# Define the input and output matrices
A = np.random.random(size=(WIDTH, WIDTH)).astype('float32')  
B = np.random.random(size=(WIDTH, WIDTH)).astype('float32')  
C = np.zeros_like(A)  # Output matrix C initialized to zero, same shape as A

In [38]:
# Allocate and transfer data to the device
d_A = cuda.to_device(A)  # Copy input matrix A to device memory
d_B = cuda.to_device(B)  # Copy input matrix B to device memory
d_C = cuda.device_array_like(d_A)  # Allocate device array for output matrix C, same shape as A

In [39]:
# Define the size of the grid
threads_per_block = (THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y)  # Threads per block in x and y dimensions

# Calculate the number of blocks needed in each dimension
blocks_per_grid_x = math.ceil(d_C.shape[0] / threads_per_block[0])  # Number of blocks along the x dimension
blocks_per_grid_y = math.ceil(d_C.shape[1] / threads_per_block[1])  # Number of blocks along the y dimension

blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)  # Tuple containing total blocks per grid

In [ ]:
# Create CUDA events to measure execution time
start = cuda.event()  # Create start event
stop = cuda.event()   # Create stop event

# Record the start time
start.record()

# Launch the CUDA kernel for matrix multiplication
matrix_multiplication[blocks_per_grid, threads_per_block](d_A, d_B, d_C)

# Record the stop time and synchronize
stop.record()  
stop.synchronize()  # Wait for the kernel to complete

# Calculate and print the elapsed time between start and stop (in milliseconds)
elapsed_time = cuda.event_elapsed_time(start, stop)
print(f"Elapsed time: {elapsed_time:.1f} ms")

In [ ]:
# Copy the result matrix from device to host
C = d_C.copy_to_host()  

# Display the resulting matrix
C

Despite the optimizations we’ve implemented so far, it's important to acknowledge that with this relatively small amount of data, the CUDA implementation of the function may still be less effective in achieving significant speedups compared to the straightforward NumPy implementation. 

CUDA excels in handling larger datasets where the overhead of kernel launches and memory transfers can be offset by the parallel processing capabilities of the GPU. However, for small matrices, the additional overhead may outweigh the benefits of parallelization.

In [42]:
# Create an empty NumPy array for the output, with the same shape as matrix A
C_np = np.empty_like(A)

In [ ]:
%%timeit -n 3 -r 3  

# Perform matrix multiplication using NumPy's matmul function
np.matmul(A, B, C_np) 

In [44]:
# mark the device memory for deletion
del d_A, d_B, d_C